# Загрузка библиотек

In [ ]:
!pip install optuna -q
!pip install catboost -q
!pip install natasha -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 81.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import lightgbm as lgb
from lightgbm import LGBMRegressor, early_stopping, Dataset, LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

import optuna
from optuna.integration import CatBoostPruningCallback

import seaborn as sns
import datetime
from wordcloud import WordCloud, STOPWORDS

import pickle

import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

sns.set(style="darkgrid")
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

from sklearn.linear_model import SGDClassifier

import gensim.downloader
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.fasttext import FastText
from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


фонтанка, лента, риа, дзен?

# Фиксируем значения

In [ ]:
RND_STATE = 7575
N_EST = 1000
sampler = optuna.samplers.TPESampler(seed=RND_STATE)

# Загрузка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Вероятно большая часть данных получена с сайта Лента.ру. Также видимо есть фонтанка. И возможно риа?

In [ ]:
class Paths:
    path_sub = '/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv'
    path_prep_test = '/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/test_prep_04_01.csv'
    path_prep_train = '/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/train_prep_04_01.csv'

paths = Paths()

# Navec

In [ ]:
# !pip install navec

In [ ]:
# !wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar

--2024-01-03 21:30:48--  https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26634240 (25M) [application/x-tar]
Saving to: ‘navec_news_v1_1B_250K_300d_100q.tar’

navec_news_v1_1B_25 100%[===================>]  25.40M  7.96MB/s    in 4.5s    

2024-01-03 21:30:54 (5.64 MB/s) - ‘navec_news_v1_1B_250K_300d_100q.tar’ saved [26634240/26634240]



In [ ]:
# from navec import Navec

# path = '/content/navec_news_v1_1B_250K_300d_100q.tar'
# navec = Navec.load(path)

In [ ]:
# HIDDEN = 300

# NewCols = ['col'+str(i) for i in range(HIDDEN)]

In [ ]:
# def get_mean_navec_vector(sentence):
#     Sum = 0
#     Count = 0

#     words = sentence.split()

#     for w in words:
#         if w in navec:
#             Sum += navec[w]
#             Count += 1

#     return Sum / Count

In [ ]:
# train = pd.read_csv(paths.path_prep_train)
# test = pd.read_csv(paths.path_prep_test)

In [ ]:
# train.shape

(39146, 7)

In [ ]:
# train_navec = train.copy()
# test_navec = test.copy()

# train_navec['vectors'] = train_navec['content_clean'].map(get_mean_navec_vector)
# test_navec['vectors'] = test_navec['content_clean'].map(get_mean_navec_vector)

In [ ]:
# train_navec[NewCols] = pd.DataFrame(train_navec['vectors'].tolist(), index=train_navec.index)
# test_navec[NewCols] = pd.DataFrame(test_navec['vectors'].tolist(), index=test_navec.index)

In [ ]:
# train_navec.drop(['vectors'], axis=1, inplace=True)
# test_navec.drop(['vectors'], axis=1, inplace=True)

In [ ]:
# train_navec.to_csv('/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/train_prep_04_01.csv', index=False)
# test_navec.to_csv('/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/test_prep_04_01.csv', index=False)

In [ ]:
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)

In [ ]:
df_test = pd.read_csv(paths.path_test)

In [ ]:
df_test.iloc[23103].content

'Тренер баскетбольной команды средней школы в американском штате Нью-Джерси подвергся жестокому избиению со стороны своих подопечных. Видео опубликовано на TMZ Sports. Инцидент произошел 4 февраля. Полиция отреагировала на сообщение о том, что у дверей средней школы избивают одного из ее работников. На видео четверо молодых людей бьют мужчину руками и ногами. Как утверждает полиция, нападавшие — игроки школьной баскетбольной команды. Жертвой стал их тренер. Полиция Нью-Джерси не стала разглашать имена подозреваемых. Мэр города Ньюарк пообещал, что ученики школы понесут наказание.'

In [ ]:
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 906.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=f5ee743c6d897d31cf03854a705310c22375554fdbc0e05ad7565f887e2acd69
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
query_embedding = model.encode('Пнекаофыжоажыф я тебя люблю?')
len(query_embedding)

768

In [ ]:
train_sentence_transformers = train.copy()
test_sentence_transformers = test.copy()

train_sentence_transformers['vectors'] = train_sentence_transformers['content_clean'].map(lambda x: model.encode(x))
test_sentence_transformers['vectors'] = test_sentence_transformers['content_clean'].map(lambda x: model.encode(x))

In [ ]:
train_sentence_transformers.head()

In [ ]:
HIDDEN = 768

NewCols = ['col_sent_trans'+str(i) for i in range(HIDDEN)]

In [ ]:
train_sentence_transformers[NewCols] = pd.DataFrame(train_sentence_transformers['vectors'].tolist(), index=train_sentence_transformers.index)
test_sentence_transformers[NewCols] = pd.DataFrame(test_sentence_transformers['vectors'].tolist(), index=test_sentence_transformers.index)

In [ ]:
train_sentence_transformers.drop(['vectors'], axis=1, inplace=True)
test_sentence_transformers.drop(['vectors'], axis=1, inplace=True)

In [ ]:
train_sentence_transformers.to_csv('/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/train_prep_05_01.csv', index=False)
test_sentence_transformers.to_csv('/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/test_prep_05_01.csv', index=False)

# Modeling

# Загрузка полностью подготовленных данных.

In [ ]:
train = pd.read_csv(paths.path_prep_train)
test = pd.read_csv(paths.path_prep_test)

In [ ]:
# Удаляем дубли
train.drop_duplicates(inplace=True)
# Удаляем пропуски
train.dropna(inplace=True)

print(f'train shape = {train.shape},  test shape = {test.shape}')
print(f'train, test is null: {train.isna().any().any(), test.isna().any().any()}')

train shape = (39146, 307),  test shape = (26275, 306)
train, test is null: (False, False)


In [ ]:
text_features = ['content', 'content_clean']
# cat_features_indxs = [1]
targets = ['topic']
# cat_cols = ['content', 'content_clean']

filtered_features = [i for i in train.columns if (i not in targets)]

## Разделение на фичи и целевую переменную.

In [ ]:
X = train[filtered_features].drop(targets, axis=1, errors="ignore")
y = train["topic"]

In [ ]:
# Разбиение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RND_STATE)

# CustomSelectColumns()

In [ ]:
class CustomSelectColumns():
    '''
    .. notes::
    Класс для вытаскивания из датасета только требуемых колонок
    '''
    def __init__(self,
                 fil_feats: list[str]=None,
                 drop: bool=False):
        self.fil_feats = fil_feats
        self.drop = drop

        # print('Init done')
    def fit(self, X: pd.DataFrame, y: pd.Series=None):
        return self

    def transform(self, X: pd.DataFrame, y: pd.Series=None) -> pd.DataFrame:
        X_ = X.copy()
        if self.drop == False:
            X_filtered = X_[self.fil_feats]
            X_filtered = pd.DataFrame(X_filtered)
            return pd.DataFrame(X_filtered)
        else:
            X_.drop(self.fil_feats, axis=1, inplace=True)
            return X_

## Базовая модель - catboost text features

### All data

In [ ]:
Catboost_pipe = Pipeline([
    # ('col_selector', CustomSelectColumns(['content', 'content_clean'], drop=True)),
    ('Catboost_simple', CatBoostClassifier(eval_metric='Accuracy',
                           iterations=500,
                           thread_count=-1,
                           random_seed=RND_STATE,
                           text_features=text_features
                                           ))
    ])

Catboost_pipe.fit(X, y,
                  Catboost_simple__text_features=text_features,
                  Catboost_simple__verbose=10,
                  Catboost_simple__early_stopping_rounds=200,
                  # Catboost_simple__eval_set=(X_val[['content']], y_val)
                  )

Learning rate set to 0.171364
0:	learn: 0.8244010	total: 8.67s	remaining: 1h 12m 7s
10:	learn: 0.8650181	total: 1m 14s	remaining: 55m 23s
20:	learn: 0.8750319	total: 2m 18s	remaining: 52m 37s
30:	learn: 0.8820058	total: 3m 20s	remaining: 50m 39s
40:	learn: 0.8863230	total: 4m 22s	remaining: 49m
50:	learn: 0.8908701	total: 5m 22s	remaining: 47m 14s
60:	learn: 0.8944209	total: 6m 23s	remaining: 46m 2s
70:	learn: 0.8976652	total: 7m 25s	remaining: 44m 49s
80:	learn: 0.9004496	total: 8m 24s	remaining: 43m 30s
90:	learn: 0.9026465	total: 9m 24s	remaining: 42m 18s
100:	learn: 0.9045624	total: 10m 20s	remaining: 40m 52s
110:	learn: 0.9062484	total: 11m 20s	remaining: 39m 45s
120:	learn: 0.9078833	total: 12m 17s	remaining: 38m 31s
130:	learn: 0.9093649	total: 13m 17s	remaining: 37m 25s
140:	learn: 0.9110765	total: 14m 14s	remaining: 36m 16s
150:	learn: 0.9122771	total: 15m 13s	remaining: 35m 10s
160:	learn: 0.9133756	total: 16m 8s	remaining: 33m 59s
170:	learn: 0.9146528	total: 17m 5s	remainin

Pipeline(steps=[('Catboost_simple',
                 <catboost.core.CatBoostClassifier object at 0x7a37d7e446d0>)])

In [ ]:
Catboost_pipe[-1].best_score_

{'learn': {'Accuracy': 0.9428549532519287, 'MultiClass': 0.16724324374805244}}

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = Catboost_pipe.predict(test)
df_sub['topic'] = preds
df_sub.to_csv('try_12.csv', index=False)

In [ ]:
# saving for blending
df_sub_collect = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = Catboost_pipe.predict(test)
df_sub_collect['Catboost_full_data'] = preds

In [ ]:
df_sub_collect.head()

,topic,index,Catboost_full_data
0,0,0,0
1,0,1,6
2,0,2,4
3,0,3,0
4,0,4,4


In [ ]:
with open(f'/content/drive/MyDrive/competitions/NLP2.0/models/Catboost_pipe_model_new.pkl', "wb") as f:
        pickle.dump(Catboost_pipe, f)

## Simple models

In [ ]:
# Для получения вектора предложения используем tf_idf
class TfidfEmbeddingVectorizer(object):
    """Get tfidf weighted vectors"""
    def __init__(self, model):
        self.word2vec = model.wv
        self.word2weight = None
        self.dim = model.vector_size

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec.get_vector(w) * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [ ]:
with open(f"/content/drive/MyDrive/competitions/NLP2.0/models/model_w2v.pkl", "rb") as f:
        model_w2v = pickle.load(f)

## Логистическая регрессия

### W2V

In [ ]:
# pipeline variant
%%time
pipe_log = Pipeline([('col_selector', CustomSelectColumns(text_features, drop=True)),
                #  ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                 ('scaler', StandardScaler()),
                 ('clf', LogisticRegression(random_state=RND_STATE,
                                            max_iter=6000,
                                            C=3,
                                            solver='lbfgs',
                                            penalty='l2',
                                            multi_class='ovr',
                                            ))])

pipe_log.fit(X, y)

print(classification_report(y_val, pipe_log.predict(X_val)))

              precision    recall  f1-score   support

           0       0.84      0.87      0.85      3374
           1       0.80      0.82      0.81       924
           2       0.84      0.76      0.80       568
           3       0.82      0.82      0.82       938
           4       0.97      0.98      0.97       835
           5       0.91      0.65      0.76        46
           6       0.98      0.97      0.98       193
           7       0.86      0.80      0.83       412
           8       0.86      0.79      0.83       540

    accuracy                           0.85      7830
   macro avg       0.88      0.83      0.85      7830
weighted avg       0.85      0.85      0.85      7830

CPU times: user 1min 13s, sys: 12.1 s, total: 1min 25s
Wall time: 1min 9s


In [ ]:
preds = pipe_log.predict(test)
df_sub_collect['log_reg_full_data'] = preds

In [ ]:
with open(f'/content/drive/MyDrive/competitions/NLP2.0/models/log_reg_pipe_model_new.pkl', "wb") as f:
        pickle.dump(pipe_log, f)

In [ ]:
# df_sub_collect

,topic,index,Catboost_full_data,log_reg_full_data
0,0,0,0,0
1,0,1,6,6
2,0,2,4,4
3,0,3,0,0
4,0,4,4,4
...,...,...,...,...
26270,0,26270,3,0
26271,0,26271,3,3
26272,0,26272,2,3
26273,0,26273,3,3


## SVM

In [ ]:
#pipeline version
%%time
pipe_svc = Pipeline([('col_selector', CustomSelectColumns(text_features, drop=True)),
                    #  ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                     ('scaler', StandardScaler()),
                     ('clf', SVC(random_state=RND_STATE,
                                 max_iter=6000,
                                 C=3,
                                 degree=4,
                                 kernel='rbf',
                                 decision_function_shape='ovo'
                            #  class_weight='balanced'
                             ))])

pipe_svc.fit(X, y)

print(classification_report(y_val, pipe_svc.predict(X_val)))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      3374
           1       0.95      0.98      0.96       924
           2       0.97      0.94      0.95       568
           3       0.96      0.97      0.97       938
           4       1.00      1.00      1.00       835
           5       1.00      0.93      0.97        46
           6       0.99      1.00      0.99       193
           7       0.98      0.99      0.98       412
           8       0.99      0.96      0.98       540

    accuracy                           0.97      7830
   macro avg       0.98      0.97      0.98      7830
weighted avg       0.97      0.97      0.97      7830

CPU times: user 4min 5s, sys: 2.69 s, total: 4min 8s
Wall time: 4min 39s


In [ ]:
preds = pipe_svc.predict(test)
df_sub_collect['svc_full_data'] = preds

In [ ]:
with open(f'/content/drive/MyDrive/competitions/NLP2.0/models/svm_pipe_model_new.pkl', "wb") as f:
        pickle.dump(pipe_svc, f)

In [ ]:
# df_sub_collect.to_csv('/content/drive/MyDrive/competitions/NLP2.0/df_sub_collect.csv', index=False)

## XGboost

In [ ]:
# init_params = {'clf__eval_metric':'accuracy',
#                'clf__enable_categorical': True,
#                'clf__booster': 'dart',
#                'clf__random_state': RND_STATE,
#                'clf__n_jobs':-1,
#                'clf__n_estimators': 200,
#                'clf__objective': 'reg:multi:softmax',
#                'clf__early_stopping_rounds': 20,
#                'clf__num_classes': 9
#               #  'callbacks': xgb.callback.LearningRateScheduler(lr_gen_simple(.04, n_est))
#                               }

# XGboost_pipe = Pipeline([('col_selector', CustomSelectColumns(text_features, drop=True)),
#                          ('scaler', StandardScaler()),
#                          ('clf', XGBClassifier(random_state=RND_STATE,
#                                               **init_params
#                              ))])

# XGboost_pipe.fit(X, y)

In [ ]:
# # функция для реализации случайного поиска
# def random_search_implementation(X, y, algorithm, init_params, grid, fit_params, cv=3, r_s=RND_STATE):
#     rgr = Pipeline([('col_selector', CustomSelectColumns(text_features, drop=True)),
#                     ('scaler', StandardScaler()),
#                     ('clf', XGBClassifier(random_state=RND_STATE,
#                                               **init_params
#                              ))])
#     grid = grid

#     model_regr = RandomizedSearchCV(estimator=rgr,
#                                     param_distributions=grid,
#                                     cv=cv,
#                                     scoring='accuracy',
#                                     n_jobs=-1,
#                                     verbose=100,
#                                     n_iter=30,
#                                     random_state=RND_STATE)

#     print('Here')

#     model_regr.fit(X, y, **fit_params)

#     print('-' * 80)
#     print(f'\n\033[1mbest parameters:\033[0m {model_regr.best_params_}')
#     print(f'\033[1mbest score =\033[0m {round(abs(model_regr.best_score_), 4)}')
#     # print(model_regr.get_params())
#     return model_regr.best_params_ | init_params

In [ ]:
# X_eval = X_val.copy()
# X_eval.drop(text_features, axis=1, inplace=True)

In [ ]:
# # функция для шага обучения
# def lr_gen_simple(init_lr, epoch):
#     decay = init_lr / epoch
#     lr_list = [init_lr/(1 + decay * i) for i in range(1, epoch + 1)]

#     return lr_list

In [ ]:
# %%time
# grid = {'clf__tree_method': ['hist', 'approx'],
#         'clf__max_depth': np.arange(9, 12, 1),
#         'clf__max_leaves': np.arange(1, 20),
#         'clf__lambda': np.arange(1, 10, 1),
#         'clf__max_bin': np.arange(10, 100, 10),
#         'clf__learning_rate': [ .05, .1, .2],
#         'clf__subsample': [.75, .85],
#         'clf__alpha': np.arange(0, 10, 1),
#         'clf__gamma': np.linspace(0, 1, 3),
#         'clf__min_child_weight': np.linspace(1, 10, 10),
#         'clf__colsample_bytree': [0.6, 0.9],
#         'clf__rate_drop':np.linspace(0.5, 1, 4),
#         'clf__skip_drop': np.linspace(0.5, 1, 4)
#         }

# init_params = {'clf__eval_metric':'accuracy',
#                'clf__enable_categorical': True,
#                'clf__booster': 'dart',
#                'clf__random_state': RND_STATE,
#                'clf__n_jobs':-1,
#                'clf__n_estimators': 500,
#                'clf__objective': 'reg:multi:softmax',
#                'clf__early_stopping_rounds': 20,
#                'clf__num_classes': 9
#               #  'clf__callbacks': [xgb.callback.LearningRateScheduler(lr_gen_simple(.04, 1000))]
#                }

# fit_params = {#'clf__eval_set': [(X_eval, y_val)],
#               'clf__verbose': 2}

# best_XGB_params = random_search_implementation(X_train, y_train,
#                                                XGboost_pipe, init_params,
#                                                grid, fit_params)

Here
Fitting 3 folds for each of 30 candidates, totalling 90 fits
--------------------------------------------------------------------------------

best parameters: {'clf__tree_method': 'hist', 'clf__subsample': 0.75, 'clf__skip_drop': 0.8333333333333333, 'clf__rate_drop': 0.8333333333333333, 'clf__min_child_weight': 3.0, 'clf__max_depth': 9, 'clf__max_bin': 60, 'clf__learning_rate': 0.2, 'clf__lambda': 3, 'clf__gamma': 0.5, 'clf__colsample_bytree': 0.6, 'clf__clf__max_leaves': 18, 'clf__alpha': 0}
best score = 0.8346
CPU times: user 2min 11s, sys: 28.7 s, total: 2min 40s
Wall time: 2h 7min 53s


In [ ]:
# best parameters: {'clf__tree_method': 'hist',
#  'clf__subsample': 0.75,
#   'clf__skip_drop': 0.8333333333333333,
#   'clf__rate_drop': 0.8333333333333333, 'clf__min_child_weight': 3.0, 'clf__max_depth': 9, 'clf__max_bin': 60, 'clf__learning_rate': 0.2, 'clf__lambda': 3, 'clf__gamma': 0.5, 'clf__colsample_bytree': 0.6, 'clf__clf__max_leaves': 18, 'clf__alpha': 0}
# best score = 0.8346
# CPU times: user 2min 11s, sys: 28.7 s, total: 2min 40s
# Wall time: 2h 7min 53s

In [ ]:
init_params = {'clf__eval_metric':'accuracy',
               'clf__enable_categorical': True,
               'clf__booster': 'dart',
               'clf__random_state': RND_STATE,
               'clf__n_jobs':-1,
               'clf__n_estimators': 200,
               'clf__objective': 'reg:multi:softmax',
               'clf__early_stopping_rounds': 20,
               'clf__num_classes': 9,
               'clf__tree_method': 'hist',
               'clf__subsample': 0.75,
               'clf__skip_drop': 0.8333333333333333,
               'clf__rate_drop': 0.8333333333333333,
               'clf__min_child_weight': 3.0,
               'clf__max_depth': 9,
               'clf__max_bin': 60,
               'clf__learning_rate': 0.2,
               'clf__lambda': 3,
               'clf__gamma': 0.5,
               'clf__colsample_bytree': 0.6,
               'clf__max_leaves': 18,
               'clf__alpha': 0
              #  'callbacks': xgb.callback.LearningRateScheduler(lr_gen_simple(.04, n_est))
                              }

XGboost_pipe = Pipeline([('col_selector', CustomSelectColumns(text_features, drop=True)),
                         ('scaler', StandardScaler()),
                         ('clf', XGBClassifier(random_state=RND_STATE,
                                              **init_params
                             ))])

XGboost_pipe.fit(X, y)

print(classification_report(y_val, XGboost_pipe.predict(X_val)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3374
           1       1.00      1.00      1.00       924
           2       1.00      1.00      1.00       568
           3       1.00      1.00      1.00       938
           4       1.00      1.00      1.00       835
           5       1.00      1.00      1.00        46
           6       1.00      1.00      1.00       193
           7       1.00      1.00      1.00       412
           8       1.00      1.00      1.00       540

    accuracy                           1.00      7830
   macro avg       1.00      1.00      1.00      7830
weighted avg       1.00      1.00      1.00      7830



In [ ]:
df_sub_collect = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/df_sub_collect.csv')

In [ ]:
# df_sub_collect

,topic,index,Catboost_full_data,log_reg_full_data,svc_full_data
0,0,0,0,0,0
1,0,1,6,6,6
2,0,2,4,4,4
3,0,3,0,0,0
4,0,4,4,4,4
...,...,...,...,...,...
26270,0,26270,3,2,3
26271,0,26271,3,3,3
26272,0,26272,2,3,3
26273,0,26273,3,3,3


In [ ]:
preds = XGboost_pipe.predict(test)
df_sub_collect['XGboost_full_data'] = preds

In [ ]:
# df_sub_collect

,topic,index,Catboost_full_data,log_reg_full_data,svc_full_data,XGboost_full_data
0,0,0,0,0,0,0
1,0,1,6,6,6,6
2,0,2,4,4,4,4
3,0,3,0,0,0,0
4,0,4,4,4,4,4
...,...,...,...,...,...,...
26270,0,26270,3,2,3,3
26271,0,26271,3,3,3,3
26272,0,26272,2,3,3,0
26273,0,26273,3,3,3,3


In [ ]:
with open(f'/content/drive/MyDrive/competitions/NLP2.0/models/XGboost_pipe.pkl', "wb") as f:
        pickle.dump(XGboost_pipe, f)

In [ ]:
# df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
# preds = XGboost_pipe.predict(test)
# df_sub['topic'] = preds
# df_sub.to_csv('try_13.csv', index=False)

## SGDClassifier

In [ ]:

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
# функция для реализации случайного поиска
def random_search_implementation(X, y, algorithm, init_params, grid, fit_params, cv=3, r_s=RND_STATE):
    rgr = Pipeline([('col_selector', CustomSelectColumns(text_features, drop=True)),
                    ('scaler', StandardScaler()),
                    ('clf', SGDClassifier(random_state=RND_STATE,
                                              **init_params
                             ))])
    grid = grid

    model_regr = RandomizedSearchCV(estimator=rgr,
                                    param_distributions=grid,
                                    cv=cv,
                                    scoring='accuracy',
                                    n_jobs=-1,
                                    verbose=100,
                                    n_iter=30,
                                    random_state=RND_STATE)

    print('Here')

    model_regr.fit(X, y, **fit_params)

    print('-' * 80)
    print(f'\n\033[1mbest parameters:\033[0m {model_regr.best_params_}')
    print(f'\033[1mbest score =\033[0m {round(abs(model_regr.best_score_), 4)}')
    # print(model_regr.get_params())
    return model_regr.best_params_ | init_params

In [ ]:
sgd ="4"

In [ ]:
%%time
grid = {'clf__loss': ['hinge',
                      'log_loss',
                      'modified_huber',
                      # 'squared_hinge',\
                      'perceptron',
                      'squared_error',
                      # 'huber',
                      # 'epsilon_insensitive',
                      # 'squared_epsilon_insensitive'
                      ],
        'clf__penalty': ['l2',
                         'l1',
                         'elasticnet',
                         ],
        'clf__alpha': [ .0001, .1, .01],
        'clf__max_iter': [1000, 3000, 6000]
        }

init_params = {#'clf__early_stopping': True,
               #'clf__random_state': RND_STATE,
               #'clf__n_jobs':-1,
               #'clf__n_iter_no_change': 10,
               #'clf__shuffle': True
               }

fit_params = {}

best_SGD_params = random_search_implementation(X_train, y_train,
                                               sgd, init_params,
                                               grid, fit_params)

Here
Fitting 3 folds for each of 30 candidates, totalling 90 fits
--------------------------------------------------------------------------------

best parameters: {'clf__penalty': 'l2', 'clf__max_iter': 1000, 'clf__loss': 'hinge', 'clf__alpha': 0.0001}
best score = 0.8045
CPU times: user 1min 23s, sys: 24.8 s, total: 1min 48s
Wall time: 1h 47min 8s


In [ ]:
# Here
# Fitting 3 folds for each of 30 candidates, totalling 90 fits
# --------------------------------------------------------------------------------

# best parameters: {'clf__penalty': 'l2', 'clf__max_iter': 1000, 'clf__loss': 'hinge', 'clf__alpha': 0.0001}
# best score = 0.8045
# CPU times: user 1min 23s, sys: 24.8 s, total: 1min 48s
# Wall time: 1h 47min 8s

In [ ]:
#pipeline version
%%time
init_params = {'penalty': 'l2', 'max_iter': 1000, 'loss': 'hinge', 'alpha': 0.0001}
pipe_SGD = Pipeline([('col_selector', CustomSelectColumns(text_features, drop=True)),
                    #  ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                     ('scaler', StandardScaler()),
                     ('clf', SGDClassifier(**init_params))])

pipe_SGD.fit(X, y)

print(classification_report(y_val, pipe_SGD.predict(X_val)))

              precision    recall  f1-score   support

           0       0.81      0.85      0.83      3374
           1       0.75      0.83      0.79       924
           2       0.82      0.68      0.75       568
           3       0.78      0.80      0.79       938
           4       0.97      0.95      0.96       835
           5       0.57      0.26      0.36        46
           6       0.94      0.89      0.91       193
           7       0.84      0.71      0.77       412
           8       0.84      0.75      0.79       540

    accuracy                           0.82      7830
   macro avg       0.81      0.75      0.77      7830
weighted avg       0.82      0.82      0.82      7830

CPU times: user 42.1 s, sys: 26.8 ms, total: 42.1 s
Wall time: 46.2 s


In [ ]:
# no  params
#  precision    recall  f1-score   support

#            0       0.81      0.85      0.83      3374
#            1       0.74      0.81      0.77       924
#            2       0.82      0.68      0.74       568
#            3       0.78      0.80      0.79       938
#            4       0.97      0.95      0.96       835
#            5       0.57      0.26      0.36        46
#            6       0.94      0.89      0.91       193
#            7       0.84      0.72      0.78       412
#            8       0.85      0.75      0.79       540

#     accuracy                           0.82      7830
#    macro avg       0.81      0.74      0.77      7830
# weighted avg       0.82      0.82      0.82      7830

# CPU times: user 48.4 s, sys: 31 ms, total: 48.4 s
# Wall time: 49.9 s

In [ ]:
preds = pipe_SGD.predict(test)
df_sub_collect['SGD_full_data'] = preds

In [ ]:
with open(f'/content/drive/MyDrive/competitions/NLP2.0/models/pipe_SGD.pkl', "wb") as f:
        pickle.dump(pipe_SGD, f)

# Stack

In [ ]:
# Некоторые модели для построение ансамбля
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier

In [ ]:
Catboost_pipe = Pipeline([
                           ('Catboost_simple', CatBoostClassifier(eval_metric='Accuracy',
                           iterations=200,
                           thread_count=-1,
                           random_seed=RND_STATE,
                           text_features=text_features))
    ])

pipe_svc = Pipeline([('col_selector', CustomSelectColumns(text_features, drop=True)),
                    #  ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                     ('scaler', StandardScaler()),
                     ('clf', SVC(random_state=RND_STATE,
                                 max_iter=6000,
                                 C=3,
                                 degree=4,
                                 kernel='rbf',
                                 decision_function_shape='ovo'
                            #  class_weight='balanced'
                             ))])

pipe_log = Pipeline([('col_selector', CustomSelectColumns(text_features, drop=True)),
                #  ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                 ('scaler', StandardScaler()),
                 ('clf', LogisticRegression(random_state=RND_STATE,
                                            max_iter=6000,
                                            C=3,
                                            solver='lbfgs',
                                            penalty='l2',
                                            multi_class='ovr',
                                            ))])

init_params = {'clf__eval_metric':'accuracy',
               'clf__enable_categorical': True,
               'clf__booster': 'dart',
               'clf__random_state': RND_STATE,
               'clf__n_jobs':-1,
               'clf__n_estimators': 200,
               'clf__objective': 'reg:multi:softmax',
               'clf__early_stopping_rounds': 20,
               'clf__num_classes': 9,
               'clf__tree_method': 'hist',
               'clf__subsample': 0.75,
               'clf__skip_drop': 0.8333333333333333,
               'clf__rate_drop': 0.8333333333333333,
               'clf__min_child_weight': 3.0,
               'clf__max_depth': 9,
               'clf__max_bin': 60,
               'clf__learning_rate': 0.2,
               'clf__lambda': 3,
               'clf__gamma': 0.5,
               'clf__colsample_bytree': 0.6,
               'clf__max_leaves': 18,
               'clf__alpha': 0
              #  'callbacks': xgb.callback.LearningRateScheduler(lr_gen_simple(.04, n_est))
                              }

XGboost_pipe = Pipeline([('col_selector', CustomSelectColumns(text_features, drop=True)),
                         ('scaler', StandardScaler()),
                         ('clf', XGBClassifier(random_state=RND_STATE,
                                              **init_params
                             ))])

pipe_SGD = Pipeline([('col_selector', CustomSelectColumns(text_features, drop=True)),
                    #  ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                     ('scaler', StandardScaler()),
                     ('clf', SGDClassifier())])

In [ ]:
with open(f"/content/drive/MyDrive/competitions/NLP2.0/models/Catboost_pipe_model_new.pkl", "rb") as f:
        Catboost_pipe = pickle.load(f)

with open(f"/content/drive/MyDrive/competitions/NLP2.0/models/log_reg_pipe_model_new.pkl", "rb") as f:
        pipe_log = pickle.load(f)

with open(f"/content/drive/MyDrive/competitions/NLP2.0/models/svm_pipe_model_new.pkl", "rb") as f:
        pipe_svc = pickle.load(f)

with open(f"/content/drive/MyDrive/competitions/NLP2.0/models/XGboost_pipe.pkl", "rb") as f:
        XGboost_pipe = pickle.load(f)

with open(f"/content/drive/MyDrive/competitions/NLP2.0/models/pipe_SGD.pkl", "rb") as f:
        pipe_SGD = pickle.load(f)

### log_reg

In [ ]:
# список базовых моделей
estimators = [("CatBoost", Catboost_pipe),
              ('SVC', pipe_svc),
              ('XGboost_pipe', XGboost_pipe),
              ('pipe_SGD', pipe_SGD),
              ('Log_Reg', pipe_log)]

# в качестве мета-модели будем использовать LogisticRegression
meta_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(verbose=False, max_iter=200),
    # n_jobs=-1,
    verbose=True,
)

stacking_classifier = meta_model
stacking_classifier

StackingClassifier(estimators=[('CatBoost',
                                Pipeline(steps=[('Catboost_simple',
                                                 <catboost.core.CatBoostClassifier object at 0x78383abac910>)])),
                               ('SVC',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x78383abac8b0>),
                                                ('scaler', StandardScaler()),
                                                ('clf',
                                                 SVC(C=3,
                                                     decision_function_shape='ovo',
                                                     degree=4, max_iter=6000,
                                                     random_state=7575))]))...
                                                 <__main__.CustomSelectColumns object at 0x78383abacaf0>),
                                                ('scaler', StandardScaler()),
                                                ('clf', SGDClassifier())])),
                               ('Log_Reg',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x78383abac9a0>),
                                                ('scaler', StandardScaler()),
                                                ('clf',
                                                 LogisticRegression(C=3,
                                                                    max_iter=6000,
                                                                    multi_class='ovr',
                                                                    random_state=7575))]))],
                   final_estimator=LogisticRegression(max_iter=200,
                                                      verbose=False),
                   verbose=True)

In [ ]:
%%time
stacking_classifier.fit(X, y)

Learning rate set to 0.371689
0:	learn: 0.8244010	total: 13.9s	remaining: 46m 11s
1:	learn: 0.8517601	total: 22s	remaining: 36m 22s
2:	learn: 0.8545701	total: 32.6s	remaining: 35m 42s
3:	learn: 0.8580442	total: 39.1s	remaining: 31m 56s
4:	learn: 0.8607776	total: 46.7s	remaining: 30m 21s
5:	learn: 0.8627701	total: 51.9s	remaining: 27m 57s
6:	learn: 0.8646350	total: 57s	remaining: 26m 10s
7:	learn: 0.8668574	total: 1m 4s	remaining: 25m 49s
8:	learn: 0.8691820	total: 1m 10s	remaining: 24m 55s
9:	learn: 0.8717366	total: 1m 16s	remaining: 24m 9s
10:	learn: 0.8729117	total: 1m 23s	remaining: 23m 50s
11:	learn: 0.8744444	total: 1m 28s	remaining: 23m 12s
12:	learn: 0.8759260	total: 1m 35s	remaining: 22m 59s
13:	learn: 0.8772799	total: 1m 41s	remaining: 22m 24s
14:	learn: 0.8788127	total: 1m 46s	remaining: 21m 51s
15:	learn: 0.8792980	total: 1m 53s	remaining: 21m 43s
16:	learn: 0.8803965	total: 1m 59s	remaining: 21m 21s
17:	learn: 0.8813161	total: 2m 4s	remaining: 21m
18:	learn: 0.8816993	total

StackingClassifier(estimators=[('CatBoost',
                                Pipeline(steps=[('Catboost_simple',
                                                 <catboost.core.CatBoostClassifier object at 0x78383abac910>)])),
                               ('SVC',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x78383abac8b0>),
                                                ('scaler', StandardScaler()),
                                                ('clf',
                                                 SVC(C=3,
                                                     decision_function_shape='ovo',
                                                     degree=4, max_iter=6000,
                                                     random_state=7575))]))...
                                                 <__main__.CustomSelectColumns object at 0x78383abacaf0>),
                                                ('scaler', StandardScaler()),
                                                ('clf', SGDClassifier())])),
                               ('Log_Reg',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x78383abac9a0>),
                                                ('scaler', StandardScaler()),
                                                ('clf',
                                                 LogisticRegression(C=3,
                                                                    max_iter=6000,
                                                                    multi_class='ovr',
                                                                    random_state=7575))]))],
                   final_estimator=LogisticRegression(max_iter=200,
                                                      verbose=False),
                   verbose=True)

In [ ]:
preds_st = stacking_classifier.predict(X_val)
balanced_accuracy_score(y_true=y_val, y_pred=preds_st)

0.7285492864763912

In [ ]:
# 0.854229941129818 - catboost + log_reg

In [ ]:
with open(f'/content/drive/MyDrive/competitions/NLP2.0/models/stacking_classifier_all_log.pkl', "wb") as f:
        pickle.dump(stacking_classifier, f)

In [ ]:
accuracy_score(y_true=y_val, y_pred=preds_st)

0.8314176245210728

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = stacking_classifier.predict(test)
df_sub['topic'] = preds
df_sub.to_csv('try_stack_all_log.csv', index=False)

### NaiveBayies

In [ ]:
# список базовых моделей
estimators = [("CatBoost", Catboost_pipe),
              ('SVC', pipe_svc),
              ('XGboost_pipe', XGboost_pipe),
              ('pipe_SGD', pipe_SGD),
              ('Log_Reg', pipe_log)]

# в качестве мета-модели будем использовать LogisticRegression
meta_model = StackingClassifier(
    estimators=estimators,
    final_estimator=MultinomialNB(),
    # n_jobs=-1,
    verbose=True,
)

stacking_classifier_NB = meta_model
stacking_classifier_NB

StackingClassifier(estimators=[('CatBoost',
                                Pipeline(steps=[('Catboost_simple',
                                                 <catboost.core.CatBoostClassifier object at 0x793405d7b9d0>)])),
                               ('SVC',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x7933fd2746d0>),
                                                ('scaler', StandardScaler()),
                                                ('clf',
                                                 SVC(C=3,
                                                     decision_function_shape='ovo',
                                                     degree=4, max_iter=6000,
                                                     random_state=7575))]))...
                                                 <__main__.CustomSelectColumns object at 0x793405f21b70>),
                                                ('scaler', StandardScaler()),
                                                ('clf', SGDClassifier())])),
                               ('Log_Reg',
                                Pipeline(steps=[('col_selector',
                                                 <__main__.CustomSelectColumns object at 0x793405b37d00>),
                                                ('scaler', StandardScaler()),
                                                ('clf',
                                                 LogisticRegression(C=3,
                                                                    max_iter=6000,
                                                                    multi_class='ovr',
                                                                    random_state=7575))]))],
                   final_estimator=MultinomialNB(), verbose=True)

In [ ]:
%%time
stacking_classifier_NB.fit(X_train, y_train)

Learning rate set to 0.169425
0:	learn: 0.8033593	total: 4.5s	remaining: 37m 26s
1:	learn: 0.8322583	total: 9.24s	remaining: 38m 20s
2:	learn: 0.8381658	total: 13.3s	remaining: 36m 35s
3:	learn: 0.8405607	total: 17.1s	remaining: 35m 20s
4:	learn: 0.8413590	total: 21.8s	remaining: 36m 1s
5:	learn: 0.8416464	total: 25.7s	remaining: 35m 12s
6:	learn: 0.8431792	total: 29.7s	remaining: 34m 52s
7:	learn: 0.8449994	total: 34.5s	remaining: 35m 23s
8:	learn: 0.8458935	total: 38.7s	remaining: 35m 9s
9:	learn: 0.8471069	total: 42.4s	remaining: 34m 39s
10:	learn: 0.8482245	total: 47.1s	remaining: 34m 54s
11:	learn: 0.8497892	total: 50.7s	remaining: 34m 21s
12:	learn: 0.8511304	total: 54.4s	remaining: 33m 56s
13:	learn: 0.8513859	total: 58.9s	remaining: 34m 5s
14:	learn: 0.8536531	total: 1m 2s	remaining: 33m 51s
15:	learn: 0.8545791	total: 1m 6s	remaining: 33m 22s
16:	learn: 0.8551858	total: 1m 10s	remaining: 33m 18s
17:	learn: 0.8571657	total: 1m 15s	remaining: 33m 31s
18:	learn: 0.8581875	total: 

KeyboardInterrupt: ignored

In [ ]:
preds_st_NB = stacking_classifier_NB.predict(X_val)
balanced_accuracy_score(y_true=y_val, y_pred=preds_st)

NotFittedError: ignored

In [ ]:
accuracy_score(y_true=y_val, y_pred=preds_st_NB)

0.9173690932311622

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = stacking_classifier_NB.predict(test)
df_sub['topic'] = preds
df_sub.to_csv('try_7.csv', index=False)

# Blending

In [ ]:
!git clone https://github.com/MLWave/Kaggle-Ensemble-Guide

Cloning into 'Kaggle-Ensemble-Guide'...
remote: Enumerating objects: 116, done.
remote: Total 116 (delta 0), reused 0 (delta 0), pack-reused 116
Receiving objects: 100% (116/116), 23.29 KiB | 1.06 MiB/s, done.
Resolving deltas: 100% (53/53), done.


# simple vote

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = Catboost_pipe.predict(test)
df_sub['topic'] = preds
df_sub = df_sub[['index', 'topic']]
df_sub.to_csv('/content/result_1.csv', index=False)

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = pipe_svc.predict(test)
df_sub['topic'] = preds
df_sub = df_sub[['index', 'topic']]
df_sub.to_csv('/content/result_2.csv', index=False)

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = pipe_log.predict(test)
df_sub['topic'] = preds
df_sub = df_sub[['index', 'topic']]
df_sub.to_csv('/content/result_3.csv', index=False)

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = XGboost_pipe.predict(test)
df_sub['topic'] = preds
df_sub = df_sub[['index', 'topic']]
df_sub.to_csv('/content/result_4.csv', index=False)

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv')
preds = pipe_SGD.predict(test)
df_sub['topic'] = preds
df_sub = df_sub[['index', 'topic']]
df_sub.to_csv('/content/result_5.csv', index=False)

In [ ]:
!python /content/Kaggle-Ensemble-Guide/src/kaggle_vote.py "/content/result_*.csv" "/content/all_blended.csv"

parsing: /content/result_2.csv
parsing: /content/result_4.csv
parsing: /content/result_1.csv
parsing: /content/result_3.csv
parsing: /content/result_5.csv
wrote to /content/all_blended.csv


In [ ]:
from collections import Counter

In [ ]:
lst_models_res = ['Catboost_full_data',	'log_reg_full_data',	'svc_full_data',	'XGboost_full_data',	'SGD_full_data']

In [ ]:
l = [1, 1, 2, 2 ,2, 2]
Counter(l).most_common(1)[0][0]

2

In [ ]:
def most_common(row):
  blanc = []
  for el in lst_models_res:
    blanc.append(row[el])
  return Counter(blanc).most_common(1)[0][0]

In [ ]:
df_sub_collect.to_csv('df_collect.csv', index=False)

In [ ]:
df_sub_collect_bl = df_sub_collect.copy()
df_sub_collect_bl['topic'] = df_sub_collect_bl.apply(most_common, axis=1)
df_sub_collect_bl = df_sub_collect_bl[['topic', 'index']]
df_sub_collect_bl

,topic,index
0,0,0
1,6,1
2,4,2
3,0,3
4,4,4
...,...,...
26270,3,26270
26271,3,26271
26272,3,26272
26273,3,26273


In [ ]:
df_sub_collect_bl.to_csv('try_14.csv', index=False)

In [ ]:
# df_bl_simple = pd.read_csv('/content/all_blended.csv')
# df_bl_simple.head()

In [ ]:
# !python /content/Kaggle-Ensemble-Guide/src/kaggle_vote.py "./content/Kaggle-Ensemble-Guide/samples/method*.csv" #"./content/Kaggle-Ensemble-Guide/kaggle_vote.csv"

Traceback (most recent call last):
  File "/content/Kaggle-Ensemble-Guide/src/kaggle_vote.py", line 7, in <module>
    loc_outfile = sys.argv[2]
IndexError: list index out of range


## weighted

In [ ]:
!python /content/Kaggle-Ensemble-Guide/src/kaggle_vote.py "/content/result_*.csv" "/content/all_blended_weighted.csv" "weighted"

parsing: /content/result_2.csv
Using weight: 1
parsing: /content/result_4.csv
Using weight: 1
parsing: /content/result_1.csv
Using weight: 1
parsing: /content/result_3.csv
Using weight: 1
parsing: /content/result_5.csv
Using weight: 1
wrote to /content/all_blended_weighted.csv


In [ ]:
df_bl_weighted = pd.read_csv('/content/all_blended_weighted.csv')
df_bl_weighted.head()

,topic,index
0,0,0
1,0,10
2,0,100
3,0,1000
4,0,10000
